# Data D4


### Data format
The data format is a .pkl file (pickle) containing a dataframe with at least these columns:
- id, a_text, b_id, b_text, c_id, c_text, research_type, y_true

In [7]:
import pandas as pd

all_papers = pd.read_csv("./data/temp/papers_1subj_2025.csv")
print(all_papers.shape, "->", all_papers.columns)

(26203, 6) -> Index(['id', 'title', 'abstract', 'categories', 'authors', 'date'], dtype='object')


In [8]:
papersB = all_papers.sample(1000, random_state=44).reset_index(drop=True)

print(papersB.shape, "->", papersB.columns)

(1000, 6) -> Index(['id', 'title', 'abstract', 'categories', 'authors', 'date'], dtype='object')


In [9]:
# Getting a single string for "paper B" and a single string for "paper C"

def concatTitleAbstract(obj):
    if obj.get('abstract', None):
        return f"{obj['title']}; {obj['abstract']}"
    return obj['title']

data = papersB.copy()

data["id"] = None
data["a_text"] = None
data["b_id"] = None
data["b_text"] = None
data["b_categories"] = None
data["c_id"] = None
data["c_text"] = None
data["c_categories"] = None

for i, row in data.iterrows():
    paper_c = papersB.loc[papersB.categories == row.categories].sample(1, random_state=i).reset_index(drop=True).loc[0]
    
    a_text = ""
    b_text = concatTitleAbstract(papersB.loc[i])
    c_text = concatTitleAbstract(paper_c)
    
    data.loc[i, "id"] = f"neg-d2-{i}"
    data.loc[i, "a_text"] = ""
    
    data.loc[i, "b_id"] = papersB.loc[i].id
    data.loc[i, "b_text"] = b_text
    data.loc[i, "b_categories"] = papersB.loc[i].categories
    
    data.loc[i, "c_id"] = paper_c.id
    data.loc[i, "c_text"] = c_text
    data.loc[i, "c_categories"] = paper_c.categories

data["y_true"] = False
data["research_type"] = None
data = data.drop(columns=["title", "abstract", "categories", "authors"])

print(data.shape, "->", data.columns)

(1000, 11) -> Index(['id', 'date', 'a_text', 'b_id', 'b_text', 'b_categories', 'c_id',
       'c_text', 'c_categories', 'y_true', 'research_type'],
      dtype='object')


In [10]:
data.to_pickle("./data/data_D4.pkl")
data = pd.read_pickle("./data/data_D4.pkl")

data

,id,date,a_text,b_id,b_text,b_categories,c_id,c_text,c_categories,y_true,research_type
0,neg-d2-0,2025-03-06,,2503.04166,On Two Parameter Time-Changed Poisson Random F...,['math.PR'],2502.13448,Criteria for asymptotic stability of eventuall...,['math.PR'],False,None
1,neg-d2-1,2025-02-26,,2502.18931,Axion Emission from Proton Cooper Pairs in Neu...,['hep-ph'],2503.1081,Next-to-leading power jet functions in the sma...,['hep-ph'],False,None
2,neg-d2-2,2025-03-03,,2503.01833,Impact of charge transfer excitons on unidirec...,['cond-mat.mes-hall'],2502.12931,Delta-function-potential junctions with quasip...,['cond-mat.mes-hall'],False,None
3,neg-d2-3,2025-02-16,,2502.11434,The blazar PKS 0605-085 as the origin of the K...,['astro-ph.HE'],2502.0115,Deciphering the Multi-Wavelength Flares of the...,['astro-ph.HE'],False,None
4,neg-d2-4,2025-03-07,,2503.05561,Test Case Generation for Dialogflow Task-Based...,['cs.SE'],2501.02875,METFORD -- Mutation tEsTing Framework fOR anDr...,['cs.SE'],False,None
...,...,...,...,...,...,...,...,...,...,...,...
995,neg-d2-995,2025-03-20,,2503.16204,Resolution of a paradox: SDSS J1257+5428 can b...,['astro-ph.SR'],2502.19256,Some musings on erythrogigantoacoustics; Obser...,['astro-ph.SR'],False,None
996,neg-d2-996,2025-02-13,,2502.09359,Flipping operators and locally harmonic Maass ...,['math.NT'],2503.03994,On families of strongly divisible modules of r...,['math.NT'],False,None
997,neg-d2-997,2025-01-08,,2501.0461,Resilient Peer-to-peer Learning based on Adapt...,['cs.LG'],2502.06738,Resurrecting saturated LLM benchmarks with adv...,['cs.LG'],False,None
998,neg-d2-998,2025-01-15,,2501.08827,CHEmical-shift selective Adiabatic Pulse (CHEA...,['physics.med-ph'],2502.03475,Calculation of a force effect from muscle acti...,['physics.med-ph'],False,None
